##### 原始数据的类型有很多，除了数字化数据外，还有大量符号化的文本数据，然而对于符号化的数据，我们是无法直接用于计算的，需要经过某些手段，将
##### 文本转换为特征向量。

##### 有些符号化的数据特征已经相对结构化，且以字典数据结构进行存储。这是我们可以用DictVectorizer对特征进行抽取和向量化

### 1.使用DictVectorizer对字典存储的数据进行特征抽取和向量化

In [1]:
dic_list = [{'name':'regan','age':26},{'name':'tingting','age':24}]
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer()
#特征转换
print(vec.fit_transform(dic_list).toarray())

[[ 26.   1.   0.]
 [ 24.   0.   1.]]


In [2]:
#输出各个维度的含义
print(vec.get_feature_names())

['age', 'name=regan', 'name=tingting']


###### 从上面的输出可以看到，DictVectorizer对于类型数据和数值数据特征数据的处理方式有很大的差异，由于类别型特征无法直接数字化表示，
###### 因此需要借助原特征的名称，组合产生新的特征，并采用0/1二值化方式进行量化。而数字化特征转换相对方便，维持原特征即可
###### 对于另外的一些存储的更加原始的数据，几乎没有使用特殊的存储结构来做存储，只是一系列的字符串，处理这些数据比较常用的文本处理方法为词袋法
###### bog of words。顾名思义，不考虑词语的出现顺序，只是将训练文本中每个出现过的词汇单独视为一列特征，我们称这些不重合的词汇的集合为词表###### vocabulary。因此对于每条训练文本，都可以在高维度的次表上面映射出一个特征向量。而特征数值的常见计算方式有两种：1.CountVectorizer和###### TfidfVectorizer。对于每一条训练样本，CountVectorizer只考虑每种词汇term在当前文本中出现的频率。Term Frequency。而TfidfVectorizer除了考虑###### 每个term在当前文本中出现的频率以外，同时要关注包含这个词汇的文本条数的倒数。Inverse Document Frequency.对比之下，训练样本越多
###### TfidfVectorizer这种方式就更加有优势。

###### 因为我们计算词频的目的在于找出对于所有文本的含义更有贡献的词汇。如果一个词汇几乎在每一个文档中都有出现，说明这是一个常用词汇，
###### 反而对模型的分类没有任何的贡献。在训练文本较多的时候，利用TfidfVectorizer压制常用词汇对分类决策的干扰，往往可以提升模型的性能。

##### 在文本挖掘中，常用的词汇如中文的你我他，英文的the,a等被称之为停用词stop words,这些停用词在文本特征的抽取中，经常以黑名单的方式过滤掉
##### 去掉停用词往往能提升模型的性能

### 1.使用CountVectorizer对文本进行特征抽取，使用朴素贝叶斯模型进行文本分类【不去除停用词】

In [5]:
from sklearn.datasets import fetch_20newsgroups
news = fetch_20newsgroups(subset='all')

In [6]:
#分割数据
from sklearn.cross_validation import train_test_split
x_train,x_test,y_train,y_test = train_test_split(news.data,news.target,test_size=0.25,random_state=123)

D:\softwares\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
count_vec = CountVectorizer()
#只是用词频的方式对训练数据和测试数据进行向量化特征抽取
x_train_vec = count_vec.fit_transform(x_train)
x_test_vec = count_vec.transform(x_test)

In [9]:
x_train_vec

<14134x145769 sparse matrix of type '<class 'numpy.int64'>'
	with 2214714 stored elements in Compressed Sparse Row format>

In [10]:
#导入朴素贝叶斯分类器
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(x_train_vec,y_train)
mnb_predict = mnb.predict(x_test_vec)

In [12]:
#查看模型性能
print('朴素贝叶斯Accuracy : %s'%(mnb.score(x_test_vec,y_test)))

朴素贝叶斯Accuracy : 0.850594227504


In [13]:
#使用classification_report计算性能指标，混淆矩阵，准确率，精确率，召回率
from sklearn.metrics import classification_report
print("朴素贝叶斯文本分类性能指标：\n",classification_report(mnb_predict,y_test,target_names=news.target_names))

朴素贝叶斯文本分类性能指标：
                           precision    recall  f1-score   support

             alt.atheism       0.91      0.83      0.87       185
           comp.graphics       0.88      0.63      0.73       326
 comp.os.ms-windows.misc       0.15      0.95      0.26        38
comp.sys.ibm.pc.hardware       0.84      0.69      0.76       293
   comp.sys.mac.hardware       0.85      0.93      0.89       228
          comp.windows.x       0.90      0.72      0.80       301
            misc.forsale       0.70      0.94      0.80       185
               rec.autos       0.93      0.88      0.90       245
         rec.motorcycles       0.92      0.98      0.95       256
      rec.sport.baseball       0.95      0.94      0.95       247
        rec.sport.hockey       0.96      0.97      0.96       240
               sci.crypt       0.98      0.80      0.88       287
         sci.electronics       0.82      0.88      0.85       231
                 sci.med       0.94      0.95      0.94    

In [30]:
## 上面的输出结果性能在86%左右

### 使用TfidfVectorizer对文本进行量化，和CountVectorizer进行性能对比

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf_x_trian = tfidf.fit_transform(x_train)
tfidf_x_test = tfidf.transform(x_test)

In [15]:
#使用朴素贝叶斯模型进行预测
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(tfidf_x_trian,y_train)
mnb_predict_tfidf = mnb.predict(tfidf_x_test)

In [16]:
#性能度量
#查看模型性能
print('TfidfVectorizer 朴素贝叶斯 Accuracy : %s'%(mnb.score(tfidf_x_test,y_test)))

TfidfVectorizer 朴素贝叶斯 Accuracy : 0.847198641766


In [17]:
from sklearn.metrics import classification_report
print("朴素贝叶斯文本分类性能指标：\n",classification_report(mnb_predict_tfidf,y_test,target_names=news.target_names))

朴素贝叶斯文本分类性能指标：
                           precision    recall  f1-score   support

             alt.atheism       0.80      0.82      0.81       165
           comp.graphics       0.77      0.87      0.82       208
 comp.os.ms-windows.misc       0.86      0.85      0.85       244
comp.sys.ibm.pc.hardware       0.85      0.78      0.81       262
   comp.sys.mac.hardware       0.83      0.95      0.88       215
          comp.windows.x       0.83      0.94      0.88       211
            misc.forsale       0.68      0.95      0.79       179
               rec.autos       0.92      0.85      0.89       252
         rec.motorcycles       0.94      0.97      0.95       264
      rec.sport.baseball       0.95      0.94      0.94       247
        rec.sport.hockey       0.98      0.92      0.95       257
               sci.crypt       0.99      0.67      0.80       346
         sci.electronics       0.81      0.89      0.85       226
                 sci.med       0.88      0.98      0.93    

In [52]:
### 从结果来看CountVectorizer和TfidfVectorizer性能相当，但TFIDFVectorizer在多文档文本分类中是被首先推荐使用的

In [53]:
#上面的测试都没有去掉停用词，那我们去掉停用词在此对比两种文本特征向量化的方式对文本分类的影响

In [18]:
from sklearn.datasets import fetch_20newsgroups
news = fetch_20newsgroups(subset='all')
#分割数据
from sklearn.cross_validation import train_test_split
x_train,x_test,y_train,y_test = train_test_split(news.data,news.target,test_size=0.25,random_state=123)

In [19]:
#使用CountVectorizer
count_vec = CountVectorizer(analyzer='word',stop_words='english')
#使用TfidfVectorizer
tfidf_vec = TfidfVectorizer(analyzer='word',stop_words='english')

In [20]:
#对训练样本和测试样本进行向量处理
x_train_count = count_vec.fit_transform(x_train)
x_test_count = count_vec.transform(x_test)

x_train_tfidf = tfidf_vec.fit_transform(x_train)
x_test_tfidf = tfidf_vec.transform(x_test)

In [21]:
#用朴素贝叶斯模型进行分类
mnb_count = MultinomialNB()
mnb_count.fit(x_train_count,y_train)
mnb_count_predict = mnb_count.predict(x_test_count)

mnb_tfidf = MultinomialNB()
mnb_tfidf.fit(x_train_tfidf,y_train)
mnb_tfidf_predict = mnb_tfidf.predict(x_test_tfidf)

In [24]:
#性能
print('CountVectorizer 朴素贝叶斯模型 Accuracy: %s'%(mnb_count.score(x_test_count,y_test)))

CountVectorizer 朴素贝叶斯模型 Accuracy: 0.873302207131


In [25]:
print('TfidfVectorizer 朴素贝叶斯模型 Accuracy: %s'%(mnb_tfidf.score(x_test_tfidf,y_test)))

TfidfVectorizer 朴素贝叶斯模型 Accuracy: 0.879244482173


In [26]:
print("CountVectorizer :\n",classification_report(mnb_count_predict,y_test,target_names=news.target_names))

print("TfidfVectorizer :\n",classification_report(mnb_tfidf_predict,y_test,target_names=news.target_names))

CountVectorizer :
                           precision    recall  f1-score   support

             alt.atheism       0.94      0.84      0.89       190
           comp.graphics       0.89      0.66      0.76       315
 comp.os.ms-windows.misc       0.26      0.95      0.41        66
comp.sys.ibm.pc.hardware       0.86      0.71      0.78       292
   comp.sys.mac.hardware       0.86      0.92      0.89       231
          comp.windows.x       0.90      0.73      0.80       297
            misc.forsale       0.80      0.91      0.85       218
               rec.autos       0.92      0.89      0.91       241
         rec.motorcycles       0.93      0.97      0.95       262
      rec.sport.baseball       0.95      0.95      0.95       245
        rec.sport.hockey       0.98      0.95      0.96       249
               sci.crypt       0.97      0.92      0.94       250
         sci.electronics       0.86      0.90      0.88       236
                 sci.med       0.94      0.95      0.95 

In [ ]:
#去掉停用词之后，可以看到TfidfVectorizer这种向量化方式性能为87.9%,高于CountVectorizer，去掉停用词之后平均性能提高2%~5%